<a href="https://colab.research.google.com/github/KatrinFrimus/Introduction-to-cloud-computing/blob/main/HW3_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
!pip install firebase
!apt-get install wkhtmltopdf
!pip install pandas matplotlib seaborn
!wkhtmltopdf table.html table.pdf
!pip install nltk


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wkhtmltopdf is already the newest version (0.12.6-2).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
Loading page (1/2)
Printing pages (2/2)                                               
Done                                                           


In [140]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import json
from google.colab import drive
from os import name
from firebase import firebase
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from google.colab import files
import seaborn as sns
from collections import Counter
from nltk.chat.util import Chat, reflections
import re

In [141]:
def split_time(record):
    dt = datetime.datetime.strptime(record["Time"], "%Y-%m-%d %H:%M:%S")
    record["Date"] = dt.date().strftime("%Y-%m-%d")
    record["Time"] = dt.time().strftime("%H:%M:%S")
    return record


In [142]:
firebase = firebase.FirebaseApplication('https://projectgroupbearcloud-default-rtdb.firebaseio.com/', None)
#result = firebase.post('/ProjectGroupBearCloud/',z)
result = firebase.get('/ProjectGroupBearCloud/-O2ExVKk-qC2MaCBpJbK', None)
df=pd.DataFrame(result)
df['Time'] = pd.to_datetime(df['Time'])

# Extract the hour from the 'Time' column
df['Hour'] = df['Time'].dt.hour


<ipython-input-142-121ce762180f>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time'])


In [143]:
def create_graph_pdfs(button):
    global fig
    if fig:
        fig.savefig('graph.pdf')
        files.download('graph.pdf')

def create_pdfs_table(button):
    global filtered_df

    filtered_df.to_html('table.html')

    !wkhtmltopdf table.html table.pdf

    files.download('table.pdf')

In [144]:
def build_graph_by_num_of_operation(change):
    global fig
    clear_output(wait=True)
    display(w_num_of_operations_by_filter)
    display(button_create_pdf_num_of_operations_by_filter)

    selected_value = change.new
    if selected_value and selected_value != 'Choose a filter':
        filter_value = df[selected_value].value_counts()
        fig, ax = plt.subplots()
        filter_value.plot(kind='bar', ax=ax)
        ax.set_title('Number of Operations by ' + selected_value)
        ax.set_xlabel(selected_value)
        ax.set_ylabel('Number of Operations')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    display(back_button)


In [145]:
def update_value_options(change):
    selected_key = w_table_filter.value
    w_table_value.options = df[selected_key].unique() if selected_key else []

def display_filtered_table(change):
    global filtered_df
    selected_key = w_table_filter.value
    selected_value =  w_table_value.value

    if selected_key and selected_value:

        try:
            filtered_df = df[df[selected_key] == selected_value]
        except Exception as e:
            print(f"Error: {e}")
            filtered_df = df
    else:
        filtered_df = df


    styled_table = filtered_df.style.set_table_styles(
        [{'selector': 'th',
          'props': [('background-color', '#f7f7f9'),
                    ('color', 'black'),
                    ('font-weight', 'bold')]},
         {'selector': 'tr:nth-child(even)',
          'props': [('background-color', '#f2f2f2')]}]
    ).set_properties(**{
        'border': '1px solid black',
        'padding': '10px',
        'text-align': 'center'
    })

    clear_output(wait=True)
    display(widgets.VBox([w_table_filter, w_table_value, button_table_create_pdf]))
    display(styled_table)
    display(back_button)

In [146]:
def plot_student_hours(selected_student):
    clear_output(wait=True)
    display(w_graph_working_hours_choose_student)
    # Filter the DataFrame for the selected student
    if selected_student == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['User'] == selected_student]

    # Count occurrences per hour
    hourly_counts = filtered_df.groupby(['Hour']).size().reindex(range(24), fill_value=0)

    # Plotting column chart
    plt.figure(figsize=(12, 8))  # Set figure size to 12x8 inches
    sns.barplot(x=hourly_counts.index, y=hourly_counts.values)

    if selected_student == 'All':
        plt.title(f'Average Working Hours Distribution for All Students')
    else:
        plt.title(f'Working Hours Distribution for {selected_student}')

    plt.xlabel('Hour of the Day')
    plt.ylabel('Number of Activities')
    plt.xticks(range(24))  # Show all hours from 0 to 23
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()

    # Print the total hours worked by the selected student
    total_hours = hourly_counts.sum()
    #print(f"Total activities recorded by {selected_student}: {total_hours}")
    display(back_button)

In [147]:
def activity():
    print(123)
    #TO_DO!!!!

In [148]:
def activity_hours_per_student():
    clear_output(wait=True)
    # Apply the new categorization function
    df['DetailedActivity'] = df['Description'].apply(detailed_categorize_activity)

    # Group by user and hour to see detailed activity patterns
    hourly_detailed_activity = df.groupby(['User', 'Hour', 'DetailedActivity']).size().unstack(fill_value=0)

    # Group by user and date to see detailed daily patterns
    daily_detailed_activity = df.groupby(['User', 'DetailedActivity']).size().unstack(fill_value=0)

    # Plotting detailed hourly activity patterns
    plt.figure(figsize=(14, 10))
    sns.heatmap(hourly_detailed_activity, cmap="YlGnBu", cbar=True, annot=True, fmt="d", linewidths=.5)
    plt.title('Detailed Hourly Activity Patterns per User')
    plt.xlabel('Hour of the Day')
    plt.ylabel('User')
    plt.show()


In [149]:
def overall_activity_by_hours():
  clear_output(wait=True)
  df['DetailedActivity'] = df['Description'].apply(detailed_categorize_activity)
  detailed_activity_patterns = df.groupby(['Hour', 'DetailedActivity']).size().unstack(fill_value=0)
  # Plotting detailed overall activity patterns by hour
  plt.figure(figsize=(14, 8))
  detailed_activity_patterns.plot(kind='bar', stacked=True, figsize=(14, 8), linewidth=0)
  plt.title('Detailed Overall Activity Patterns by Hour')
  plt.xlabel('Hour of the Day')
  plt.ylabel('Number of Activities')
  plt.legend(title='Activity Type')
  plt.grid(True)
  plt.tight_layout()
  plt.show()

In [150]:
def detailed_categorize_activity(description):
    description = description.lower()
    if 'create' in description:
        return 'Create'
    elif 'delete' in description:
        return 'Delete'
    elif 'tab' in description:
        return 'Tab Change'
    elif 'edit' in description:
        return 'Edit'
    elif 'open' in description:
        return 'Open'
    elif 'close' in description:
        return 'Close'
    elif 'hide' in description or 'show' in description:
        return 'Hide/Show'
    elif 'change' in description:
        return 'Change Element'
    elif 'update' in description:
        return 'Update'
    elif 'import' in description or 'export' in description:
        return 'Import/Export'
    elif 'insert' in description:
        return 'Insert'
    elif 'commit' in description or 'start' in description:
        return 'Commit/Start'
    elif 'cancel' in description or 'undo' in description:
        return 'Undo/Cancel'
    elif 'generic' in description or 'other' in description:
        return 'Other'
    else:
        return 'Miscellaneous'

In [151]:
def reset_interface(b):
    clear_output(wait=True)  # Clear the output to reset the interface
    display(widgets.HTML("<h1>Welcome to On-Shape Analyzer</h1>"))
    display(button_num_of_operations_by_filter)
    display(button_create_table)
    display(button_graph_workin_hours)
    display(button_hourly_activity_patterns)
    display(button_second_graph)
    display(button_overAll_acctivity_patterns)
    display(chat_bot)

In [152]:
def show_widgets(b):
    if (b.description == "Graph of number of operations by filter"):
      clear_output(wait=True)  # Clear the output to avoid duplication
      display(widgets.HTML("<h1>Welcome to On-Shape Analyzer</h1>"))
      display(w_num_of_operations_by_filter)
      display(button_create_pdf_num_of_operations_by_filter)
      display(back_button)
    if (b.description == "create table"):
      clear_output(wait=True)  # Clear the output to avoid duplication
      display(widgets.HTML("<h1>Welcome to On-Shape Analyzer</h1>"))
      display(w_table_filter)
      display(w_table_value)
      display(button_table_create_pdf)
      display(back_button)
    if (b.description == "Graph of working hours per students"):
      clear_output(wait=True)  # Clear the output to avoid duplication
      display(widgets.HTML("<h1>Welcome to On-Shape Analyzer</h1>"))
      display(w_graph_working_hours_choose_student)
      display(back_button)
    if (b.description == "hourly activity paterns per user"):
      clear_output(wait=True)  # Clear the output to avoid duplication
      display(widgets.HTML("<h1>Welcome to On-Shape Analyzer</h1>"))
      activity_hours_per_student()
      display(back_button)
    if (b.description == "Over all activity patterns per hour"):
      clear_output(wait=True)  # Clear the output to avoid duplication
      display(widgets.HTML("<h1>Welcome to On-Shape Analyzer</h1>"))
      overall_activity_by_hours()
      display(back_button)
    if (b.description == "second graph"):
      clear_output(wait=True)  # Clear the output to avoid duplication
      activity()
      display(back_button)
    if (b.description == "Chat bot"):
      clear_output(wait=True)  # Clear the output to avoid duplication
      display(widgets.HTML("<h1>Welcome to On-Shape Analyzer</h1>"))
      chat_bot()
      display(back_button)


In [156]:
display(widgets.HTML("<h1>Welcome to On-Shape Analyzer</h1>"))#title
#back buttom
back_button = widgets.Button(description="back",
    style={'font_size': '20px', 'font_weight': 'bold'},
    layout=widgets.Layout(width='100px', height='30px', border='2px solid red')
)

back_button.on_click(reset_interface)



#first graph
button_num_of_operations_by_filter = widgets.Button(
    description="Graph of number of operations by filter",
    style={'font_size': '40px', 'font_weight': 'bold'},
    layout=widgets.Layout(width='300px', height='50px', border='2px solid black')
)
button_num_of_operations_by_filter.on_click(show_widgets)

w_num_of_operations_by_filter = widgets.Dropdown(
    options=['Choose a filter', 'Time', 'Document', 'Tab', 'User', 'Date'],
    value='Choose a filter',
    description='Filter:',
    layout=widgets.Layout(width='300px', height='50px')
)
button_create_pdf_num_of_operations_by_filter = widgets.Button(
    description="Create PDF",
    layout=widgets.Layout(width='150px', height='50px', border='2px solid blue')
)
button_create_pdf_num_of_operations_by_filter.on_click(create_graph_pdfs)
w_num_of_operations_by_filter.observe(build_graph_by_num_of_operation, names='value')

display(button_num_of_operations_by_filter)


#table
button_create_table = widgets.Button(
    description="create table",
    style={'font_size': '40px', 'font_weight': 'bold'},
    layout=widgets.Layout(width='300px', height='50px', border='2px solid black')
)
button_create_table.on_click(show_widgets)

w_table_filter = widgets.Dropdown(
    options=df.columns,
    description='Select key:',
    disabled=False,
)
w_table_filter.observe(update_value_options, names='value')

w_table_value= widgets.Dropdown(
    description='Select value:',
    disabled=False,
)
w_table_value.observe(display_filtered_table, names='value')

button_table_create_pdf = widgets.Button(description="create PDF")
button_table_create_pdf.on_click(create_pdfs_table)

display(button_create_table)


#working hours
button_graph_workin_hours = widgets.Button(
    description="Graph of working hours per students",
    style={'font_size': '40px', 'font_weight': 'bold'},
    layout=widgets.Layout(width='300px', height='50px', border='2px solid black')
)
button_graph_workin_hours.on_click(show_widgets)

w_graph_working_hours_choose_student = widgets.Dropdown(
    options=['All'] + list(df['User'].unique()),
    description='Select Student:',
)
#hours_polt = widgets.interactive(plot_student_hours, selected_student=w_graph_working_hours_choose_student)
w_graph_working_hours_choose_student.observe(lambda change: plot_student_hours(change.new), names='value')
display(button_graph_workin_hours)
button_graph_workin_hours.on_click(show_widgets)



#patterns
button_hourly_activity_patterns = widgets.Button(
    description="hourly activity paterns per user",
    style={'font_size': '40px', 'font_weight': 'bold'},
    layout=widgets.Layout(width='300px', height='50px', border='2px solid black')
)
button_hourly_activity_patterns.on_click(show_widgets)
display(button_hourly_activity_patterns)

#to change
button_second_graph = widgets.Button(
    description="second graph",
    style={'font_size': '40px', 'font_weight': 'bold'},
    layout=widgets.Layout(width='300px', height='50px', border='2px solid black')
)
button_second_graph.on_click(show_widgets)
display(button_second_graph)

#patterns
button_overAll_acctivity_patterns = widgets.Button(
    description="Over all activity patterns per hour",
    style={'font_size': '40px', 'font_weight': 'bold'},
    layout=widgets.Layout(width='300px', height='50px', border='2px solid black')
)
button_overAll_acctivity_patterns.on_click(show_widgets)
display(button_overAll_acctivity_patterns)



#chat bot
button_overAll_acctivity_patterns = widgets.Button(
    description="Chat bot",
    style={'font_size': '40px', 'font_weight': 'bold'},
    layout=widgets.Layout(width='300px', height='50px', border='2px solid orange')
)
button_overAll_acctivity_patterns.on_click(show_widgets)
display(button_overAll_acctivity_patterns)

HTML(value='<h1>Welcome to On-Shape Analyzer</h1>')

Button(description='Graph of number of operations by filter', layout=Layout(border='2px solid black', height='…

Button(description='create table', layout=Layout(border='2px solid black', height='50px', width='300px'), styl…

Button(description='Graph of working hours per students', layout=Layout(border='2px solid black', height='50px…

Button(description='hourly activity paterns per user', layout=Layout(border='2px solid black', height='50px', …

Button(description='second graph', layout=Layout(border='2px solid black', height='50px', width='300px'), styl…

Button(description='Chat bot', layout=Layout(border='2px solid orange', height='50px', width='300px'), style=B…

<function __main__.chat_bot()>

In [155]:
def chat_bot():
      # Define functions to answer questions
    def get_students():
        return df['User'].unique().tolist()

    def get_most_common_actions():
        actions = df['Description'].tolist()
        return Counter(actions).most_common(2)

    def get_dates_with_most_actions():
      #problem
        df['Date'] = df['Time'].dt.date
        return df['Date'].value_counts()

    def get_actions_per_day():
      #problem
        df['Date'] = df['Time'].dt.date
        return df.groupby('Date')['Description'].count()

    def get_actions_per_student():
        return df['User'].value_counts()

    def get_top_student():
        return df['User'].value_counts().idxmax()


    def get_shortest_actions():
        global df
        df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
        df = df.sort_values(by='Time')
        df['time_diff'] = df['Time'].diff().dt.total_seconds().fillna(0)
        shortest_actions = df.nsmallest(5, 'time_diff')
        shortest_actions = shortest_actions[['Description', 'time_diff']]
        shortest_actions = shortest_actions.reset_index(drop=True)
        return shortest_actions

    def get_total_time():
        df['Time'] = pd.to_datetime(df['Time'])
        total_time = (df['Time'].max() - df['Time'].min()).total_seconds()
        return total_time

    def get_shared_actions():
        action_counts = df.groupby(['Description', 'User']).size().unstack(fill_value=0)
        shared_actions = action_counts[action_counts.gt(0).sum(axis=1) > 1]
        shared_actions_summary = shared_actions.sum(axis=1)
        top_shared_actions = shared_actions_summary.nlargest(5)
        return top_shared_actions

    def get_unique_actions_per_student():
        action_counts = df.groupby(['Description', 'User']).size().unstack(fill_value=0)
        unique_actions = action_counts[action_counts.gt(0).sum(axis=1) == 1]
        unique_actions_summary = unique_actions.sum(axis=1)
        top_unique_actions = unique_actions_summary.nsmallest(5)
        result = pd.merge(df, top_unique_actions.reset_index(), on='Description', how='inner')
        result = result[['Description', 'User']]
        return result.drop_duplicates()

    def get_actions_per_hour():
        df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
        df['Hour'] = df['Time'].dt.hour
        actions_per_hour = df.groupby('Hour')['Description'].count()
        actions_per_hour.index = actions_per_hour.index.map(lambda x: f'{x:02d}:00')
        top_5_actions_per_hour = actions_per_hour.nlargest(5)

        return top_5_actions_per_hour

    def get_night_actions():
        df['Hour'] = df['Time'].dt.hour
        night_hours = df[(df['Hour'] >= 0) & (df['Hour'] < 6)]
        return night_hours['Description'].count()

    # Define some patterns and responses
    patterns = [
        (r'who are the students working on the project\?', get_students),
        (r'What are the most common actions and how many times have they been performed?\?', get_most_common_actions),
        (r'what dates have the most actions\?', get_dates_with_most_actions),
        (r'how many actions are performed each day\?', get_actions_per_day),
        (r'how many actions are performed by each student\?', get_actions_per_student),
        (r'which student performed the most actions\?', get_top_student),
        (r'what are the shortest actions\?', get_shortest_actions),
        (r'what is the total time for all actions\?', get_total_time),
        (r'which actions were performed by more than one student\?', get_shared_actions),
        (r'what actions were performed by only one student\?', get_unique_actions_per_student),
        (r'what hours of the day have the most actions\?', get_actions_per_hour),
        (r'how many actions were performed at night\?', get_night_actions),
    ]

    # Create a chatbot
    class CustomChat(Chat):
        def respond(self, str):
            for pattern, func in self._pairs:
                match = re.search(pattern, str)
                if match:
                    response = func()
                    return response
            return "Sorry, I didn't understand that question."

    chatbot = CustomChat(patterns, reflections)

    # Define the dropdown options
    questions = [
        "who are the students working on the project?",
        "What are the most common actions and how many times have they been performed?",
        "what dates have the most actions?",
        "how many actions are performed each day?",
        "how many actions are performed by each student?",
        "which student performed the most actions?",
        "what are the shortest actions?",
        "what is the total time for all actions?",
        "which actions were performed by more than one student?",
        "what actions were performed by only one student?",
        "what hours of the day have the most actions?",
        "how many actions were performed at night?"
    ]
    print("Hello! I'm On-Shape Bot. How can I help you today?")

    # Create the dropdown widget
    dropdown = widgets.Dropdown(
        options=["--Select a question--"] + questions,
        description='Questions:',
        disabled=False,
    )

    # Create an output widget to display the response
    output = widgets.Output()

    def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            question = change['new']
            if question != "--Select a question--":
                response = chatbot.respond(question)
                with output:
                    output.clear_output()
                    print(f"Question: {question}")
                    print(f"ChatGPT: {response}")

    # Link the dropdown to the function
    dropdown.observe(on_change)

    # Display the widgets
    display(dropdown, output)
    return None